<a href="https://colab.research.google.com/github/swlee23/Deep-Learning-Time-Series-Anomaly-Detection/blob/master/DeepAnT_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Things to be included
# 0. preprocess segment for seq (raw -> input format)
# 1. visualization tools for sgd
# 2. visualization tools for plotting actual and predicted sequence, and anomaly points
# 3. Implementation of unsupervised learning techniques
# 4. GPU position check If the machine on which you train on has a GPU on 0,
#    make sure to use 0 instead of 1. You can check that by running a simple 
#    command on your terminal: for example, nvidia-smi
# 5. other optimizer such as adam is better?

In [0]:
import numpy as np
import os
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation
from keras.optimizers import SGD

In [0]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1" #model will be trained on GPU 1

In [0]:
              """Hyperparameters"""

w = 5                    # History window (length of chopped sequences) / kernel size       
p_w = 1                  # Prediction window (number of time stampes required to be 
                         # predicted)

num_filt_1 = 32          # Number of filters in first conv layer
num_filt_2 = 32          # Number of filters in second conv layer
num_nrn_dl = 40          # Number of neurons in dense layer
num_nrn_ol = p_w         # Number of neurons in output layer

conv_strides = 1
pool_size_1 = 2          # Length of window of pooling layer 1
pool_size_2 = 2          # Length of window of pooling layer 2
pool_strides_1 = 2       # Stride of window of pooling layer 1
pool_strides_2 = 2       # Stride of window of pooling layer 2

epochs = 100
batch_size = 30          # Daily current trend data for 1 month 
dropout_rate = 0.5       # Dropout rate in the fully connected layer
learning_rate = 2e-5  
anm_det_thr = 0.8        # Threshold for classifying anomaly (0.5~0.8)

In [0]:
                """Data loading"""

In [26]:
              """Data preprocessing"""
# 1. split a univariate sequence into samples
def split_sequence(raw_seq, n_steps, prediction_seq_len):
	X, Y = list(), list()
	for i in range(len(raw_seq)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(raw_seq)-prediction_seq_len:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = raw_seq[i:end_ix], raw_seq[end_ix:end_ix + prediction_seq_len]
		X.append(seq_x)
		Y.append(seq_y)
	return np.array(X), np.array(Y)

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = w
# choose length of prediction sequence
prediction_seq_len = p_w
# split into samples
X, Y = split_sequence(raw_seq, n_steps, prediction_seq_len)
# summarize the data
for i in range(len(X)):
	print(X[i], Y[i])
  

  
# 2. reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

[10 20 30 40 50] [60]
[20 30 40 50 60] [70]
[30 40 50 60 70] [80]
[40 50 60 70 80] [90]
4 1


In [0]:
              """Generate model for predictor"""
def predictor_model_generator():
  model = Sequential()
  # Input Layer
  # Reshape X to 3-D tensor: [batch_size, width, channels]
  # MNIST images are 28x28 pixels, and have one color channel


  # Convolutional Layer #1
  # Computes 32 features using a 1D filter(kernel) of with w with ReLU activation. 
  # Padding is added to preserve width.
  # Input Tensor Shape: [batch_size, input_seq_len, 1]
  # Output Tensor Shape: [batch_size, w, num_filt_1] (num_filt_1 = 32 feature vectors)
  model.add(Conv1D(filters=num_filt_1,
                   kernel_size=w,
                   strides=conv_strides,
                   padding='valid',
                   activation='relu',
                   input_shape=(w, 1)))

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, w, num_filt_1]
  # Output Tensor Shape: [batch_size, 0.25 * w, num_filt_1]
  
  model.add(MaxPooling1D(pool_size=pool_size_1, 
                         strides=pool_strides_1, 
                         padding='valid'))

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 0.25 * w, 32]
  # Output Tensor Shape: [batch_size, 0.25 * w, num_filt_1 * num_filt_2]
  model.add(Conv1D(filters=num_filt_2,
                   kernel_size=w,
                   strides=conv_strides,
                   padding='valid',
                   activation='relu'))

  # Max Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 0.25 * w, num_filt_1 * num_filt_2]
  # Output Tensor Shape: [batch_size, 0.25^2 * w, num_filt_1 * num_filt_2]
  model.add(MaxPooling1D(pool_size=pool_size_2, 
                         strides=pool_strides_2, 
                         padding='valid'))

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 0.25^2 * w, num_filt_1 * num_filt_2]
  # Output Tensor Shape: [batch_size, 0.25^2 * w * num_filt_1 * num_filt_2]
  model.add(Flatten())

  # Dense Layer (Output layer)
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 0.25^2 * w * num_filt_1 * num_filt_2]
  # Output Tensor Shape: [batch_size, 1024]
  model.add(Dense(units=num_nrn_dl, activation='relu'))  

  # Dropout
  # Prevents overfitting in deep neural networks
  model.add(Dropout(dropout_rate))

  # Output layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, p_w]
  model.add(Dense(units=num_nrn_ol))

In [36]:
                """Configure model"""
sgd = keras.optimizers.SGD(lr=learning_rate, 
                           decay=1e-6, 
                           momentum=0.9, 
                           nesterov=True)
model.compile(optimizer='sgd', 
              loss='mean_absolute_error', 
              metrics=['accuracy']) 

NameError: ignored

In [0]:
                    """Training"""
# Train the model
model.fit(
  train_seqs,
  to_categorical(train_labels),
  epochs=epochs,
  batch_size=batch_size,
)

In [0]:
                    """Testing"""
# Returns the loss value & metrics values for the model in test mode
model.evaluate(
    test_seq,
    to_categorical(test_labels)
) 

In [0]:
                  """Save Weights"""
# save it to disk so we can load it back up anytime
model.save_weights('model.h5')  

In [0]:
                   """Predicting"""
# Build model 
DeepAnT_model_generator()
          
# Load the model's saved weights.
model.load_weights('model.h5')
          
# Predict on the first 5 test images.
predictions = model.predict(test_seqs[:5])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(ground_truth_seqs[:5]) # [7, 2, 1, 0, 4]


In [0]:
                """Anomaly detector"""
def anomaly_detector(prediction_seq, ground_truth_seq)
  # calculate Euclidean between actual seq and predicted seq
  dist = np.linalg.norm(ground_truth_seq - prediction_seq)  
  if (dist > anm_det_thr)
    return true  # anomaly
  else
    return false # normal 